In [1]:
import duckdb
import pandas as pd
import os
import csv

In [2]:
# open connection to the database
con = duckdb.connect('/workspaces/practice/tuva.db')

In [3]:
# create schemas for syntegra_ehr and syntegra_claims
con.execute('CREATE SCHEMA IF NOT EXISTS syntegra_ehr')
con.execute('CREATE SCHEMA IF NOT EXISTS syntegra_claims')

In [4]:
# for each file in the directory /workspaces/practice/syntegra/claims, create a table in the syntegra_claims schema
for file in os.listdir('/workspaces/practice/syntegra/claims'):
    if file.endswith('.csv'):
        con.execute('CREATE or replace table syntegra_claims.' + file[:-4] + ' AS SELECT * FROM read_csv_auto(\'/workspaces/practice/syntegra/claims/' + file + '\')')

In [5]:
# for each file in the directory /workspaces/practice/syntegra/ehr, create a table in the syntegra_ehr schema
for file in os.listdir('/workspaces/practice/syntegra/ehr'):
    if file.endswith('.csv'):
        con.execute('CREATE or replace table syntegra_ehr.' + file[:-4] + ' AS SELECT * FROM read_csv_auto(\'/workspaces/practice/syntegra/ehr/' + file + '\')')

In [6]:
con.execute('alter table syntegra_ehr.location rename column column0 to facility_npi')
con.execute('alter table syntegra_ehr.location rename column column1 to facility_name')
con.execute('alter table syntegra_ehr.location rename column column2 to facility_type')
con.execute('alter table syntegra_ehr.location rename column column3 to hospital_type')
con.execute('alter table syntegra_ehr.location rename column column4 to parent_organization')
con.execute('alter table syntegra_ehr.location rename column column5 to data_source')

con.execute('alter table syntegra_ehr.practitioner rename column column0 to physician_npi')
con.execute('alter table syntegra_ehr.practitioner rename column column1 to name')
con.execute('alter table syntegra_ehr.practitioner rename column column2 to specialty')
con.execute('alter table syntegra_ehr.practitioner rename column column3 to sub_specialty')
con.execute('alter table syntegra_ehr.practitioner rename column column4 to data_source')

In [7]:
# show all tables in the database
con.execute('select * from information_schema.tables').fetchdf()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,tuva,syntegra_claims,procedure,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
1,tuva,syntegra_claims,patient,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
2,tuva,syntegra_claims,member_month,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
3,tuva,syntegra_claims,medical_claim,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
4,tuva,syntegra_claims,encounter,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
5,tuva,syntegra_claims,coverage,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
6,tuva,syntegra_claims,condition,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
7,tuva,syntegra_ehr,vital_sign,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
8,tuva,syntegra_ehr,procedure,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
9,tuva,syntegra_ehr,practitioner,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN


In [8]:
# select 5 random rows from the syntegra_claims.procedure table
con.execute('select * from syntegra_claims.condition offset floor(random() * (select count(*) from syntegra_claims.condition)) limit 5').fetchdf()

,ENCOUNTER_ID,PATIENT_ID,CONDITION_DATE,CONDITION_TYPE,CODE_TYPE,CODE,DESCRIPTION,DIAGNOSIS_RANK,PRESENT_ON_ADMIT,DATA_SOURCE
0,569d6b4fbdeea218105562bf8dc13cf6,10902,2018-10-30,discharge diagnosis,icd-10-cm,I10,Essential (primary) hypertension,5,,cclf
1,51841a8a75306e97d246fadda8a50318,12330,2016-02-24,discharge diagnosis,icd-10-cm,I2510,Athscl heart disease of native coronary artery...,2,,cclf
2,5ca6c559315b9a2a48498bc84d726919,10472,2018-06-04,discharge diagnosis,icd-10-cm,Z87891,Personal history of nicotine dependence,8,,cclf
3,f98cc9103416d2f2c8a7810da08081e8,10321,2018-07-21,discharge diagnosis,icd-10-cm,F5101,Primary insomnia,2,,cclf
4,aec76d04b08a713b7e341ec9a34eb477,117,2016-06-13,discharge diagnosis,icd-10-cm,Z0000,Encntr for general adult medical exam w/o abno...,1,,cclf


In [9]:
con.execute('select * from syntegra_claims.coverage offset floor(random() * (select count(*) from syntegra_claims.coverage)) limit 5').fetchdf()

,PATIENT_ID,COVERAGE_START_DATE,COVERAGE_END_DATE,PAYER,PAYER_TYPE,DATA_SOURCE
0,10439,2016-01-01,2018-09-01,medicare,medicare,cclf
1,10763,2016-01-01,2018-09-01,medicare,medicare,cclf
2,11312,2016-01-01,2018-09-01,medicare,medicare,cclf
3,11733,2016-01-01,2018-09-01,medicare,medicare,cclf
4,12076,2016-01-01,2018-07-01,medicare,medicare,cclf


In [10]:
con.execute('select * from syntegra_claims.encounter offset floor(random() * (select count(*) from syntegra_claims.encounter)) limit 5').fetchdf()

,ENCOUNTER_ID,PATIENT_ID,ENCOUNTER_TYPE,ENCOUNTER_START_DATE,ENCOUNTER_END_DATE,ADMIT_SOURCE_CODE,ADMIT_SOURCE_DESCRIPTION,ADMIT_TYPE_CODE,ADMIT_TYPE_DESCRIPTION,DISCHARGE_DISPOSITION_CODE,DISCHARGE_DISPOSITION_DESCRIPTION,PHYSICIAN_NPI,LOCATION,FACILITY_NPI,MS_DRG,PAID_AMOUNT,CHARGE_AMOUNT,DATA_SOURCE
0,c31ba7c51541905a1d92d998ebe47e28,12959,other,2017-06-01,2017-06-01,,,,,,,1497933162,,,,145.77,164.92,cclf
1,cec2297071bdcc9d8df754a1d94c3f57,12272,office visit,2017-05-13,2017-05-13,,,,,,,1598794026,,,,92.01,116.15,cclf
2,7e351098e2a7b937f8835d9fba82b9a3,10059,office visit,2018-03-25,2018-03-25,,,,,,,1184629321,,,,11.91,11.74,cclf
3,377337d0cd35cb6c95cc7b98e873cd46,11923,other,2018-07-03,2018-07-03,,,,,,,1932166386,,,,18.28,17.85,cclf
4,66a757dd7e6fc8a28823113680d4832e,10665,other,2016-09-28,2016-09-28,,,,,,,1255314704,,,,132.05,135.93,cclf


In [11]:
con.execute('select * from syntegra_claims.medical_claim offset floor(random() * (select count(*) from syntegra_claims.medical_claim)) limit 5').fetchdf()

,CLAIM_ID,CLAIM_LINE_NUMBER,CLAIM_TYPE,ENCOUNTER_ID,PATIENT_ID,CLAIM_START_DATE,CLAIM_END_DATE,CLAIM_LINE_START_DATE,CLAIM_LINE_END_DATE,BILL_TYPE_CODE,...,HCPCS_MODIFIER_3,HCPCS_MODIFIER_4,HCPCS_MODIFIER_5,RENDERING_NPI,BILLING_NPI,FACILITY_NPI,DISCHARGE_DISPOSITION_CODE,DISCHARGE_DISPOSITION_DESCRIPTION,CHARGE_AMOUNT,DATA_SOURCE
0,bd00353b04060f38252159e73b22f6aa,1,P,bd00353b04060f38252159e73b22f6aa,12282,2017-03-05,2017-03-05,2017-03-05,2017-03-05,,...,,,,1003802547,,,,,76.61,cclf
1,bd00353b04060f38252159e73b22f6aa,2,P,bd00353b04060f38252159e73b22f6aa,12282,2017-03-05,2017-03-05,2017-03-05,2017-03-05,,...,,,,1003802547,,,,,30.45,cclf
2,146c7a3e5fe9861adb69de1bf6749133,1,P,f2541173f2ac29763c9099a18f3e78ee,12277,2016-04-16,2016-04-16,2016-04-16,2016-04-16,,...,,,,1003899907,,,,,126.01,cclf
3,4e648fae1922025f95ee705a17f7c76d,1,P,4e648fae1922025f95ee705a17f7c76d,12445,2018-02-10,2018-02-10,2018-02-10,2018-02-10,,...,,,,1235225558,,,,,0.00,cclf
4,4e648fae1922025f95ee705a17f7c76d,2,P,4e648fae1922025f95ee705a17f7c76d,12445,2018-02-10,2018-02-10,2018-02-10,2018-02-10,,...,,,,1235225558,,,,,0.00,cclf


In [12]:
con.execute('select * from syntegra_claims.member_month offset floor(random() * (select count(*) from syntegra_claims.member_month)) limit 5').fetchdf()

,PATIENT_ID,PAYER,MONTH,YEAR,DUAL_STATUS,MEDICARE_STATUS,DATA_SOURCE
0,10186,medicare,12,2016,,20,cclf
1,10186,medicare,2,2017,,20,cclf
2,10186,medicare,3,2017,,20,cclf
3,10186,medicare,4,2017,,20,cclf
4,10186,medicare,5,2017,,20,cclf


In [13]:
con.execute('select * from syntegra_claims.patient offset floor(random() * (select count(*) from syntegra_claims.patient)) limit 5').fetchdf()

,PATIENT_ID,GENDER,BIRTH_DATE,RACE,ZIP_CODE,STATE,COUNTY,DECEASED_FLAG,DEATH_DATE,DATA_SOURCE
0,11030,male,1935-10-09,other,,Maryland,Montgomery,1,2018-08-31,cclf
1,12718,male,1951-02-23,white,,Pennsylvania,Beaver,0,,cclf
2,12139,male,1942-08-22,white,,Illinois,Du Page,0,,cclf
3,13103,female,1935-06-10,black,,Nevada,Clark,0,,cclf
4,12876,female,1929-05-26,white,,Alabama,Jefferson,0,,cclf


In [14]:
con.execute('select * from syntegra_claims.procedure offset floor(random() * (select count(*) from syntegra_claims.procedure)) limit 5').fetchdf()

,ENCOUNTER_ID,PATIENT_ID,PROCEDURE_DATE,CODE_TYPE,CODE,DESCRIPTION,PHYSICIAN_NPI,DATA_SOURCE
0,472eaa35d28111222178ef91d95ca27e,12288,2016-02-27,icd-10-pcs,0WUF4JZ,"Supplement Abdominal Wall with Synth Sub, Perc...",1003809963,cclf
1,2387e1845d052f4ebd1d40cfb5699138,1296,2017-08-29,icd-10-pcs,0SB20ZZ,"Excision of Lumbar Vertebral Disc, Open Approach",1518170927,cclf
2,8d39b9b210059ff8741ea8f0c88ab3a0,11170,2017-03-20,icd-10-pcs,4A023N7,"Measure of Cardiac Sampl & Pressure, L Heart, ...",1649251281,cclf
3,20c8486857e56e7d503aa58cf01ddbe5,12140,2016-07-19,icd-10-pcs,4A023N7,"Measure of Cardiac Sampl & Pressure, L Heart, ...",1265472203,cclf
4,748c0eaeeef2be0120e8c64791eae87d,11862,2016-02-14,icd-10-pcs,4A023N7,"Measure of Cardiac Sampl & Pressure, L Heart, ...",1255309977,cclf


In [15]:
con.execute('select * from syntegra_ehr.allergy offset floor(random() * (select count(*) from syntegra_ehr.allergy)) limit 5').fetchdf()

,encounter_id,patient_id,status,allergy_description,severity,data_source
0,819650,243773,active,No Known Allergies,,Syn
1,1006864,243773,active,No Known Allergies,,Syn
2,578846,289529,active,No Known Allergies,,Syn
3,543304,156219,active,No Known Allergies,,Syn
4,928946,14873,active,Sulfa (Sulfonamide Antibiotics),,Syn


In [16]:
con.execute('select * from syntegra_ehr.condition offset floor(random() * (select count(*) from syntegra_ehr.condition)) limit 5').fetchdf()

,encounter_id,patient_id,condition_date,condition_type,code_type,code,description,diagnosis_rank,present_on_admit,data_source
0,601656,162393,2018-04-08,discharge diagnosis,icd-10-cm,Z87891,Personal history of nicotine dependence,12,,Syn
1,601656,162393,2018-04-08,discharge diagnosis,icd-10-cm,R002,Palpitations,2,,Syn
2,601656,162393,2018-04-08,discharge diagnosis,icd-10-cm,E785,"Hyperlipidemia, unspecified",3,,Syn
3,601656,162393,2018-04-08,discharge diagnosis,icd-10-cm,I10,Essential (primary) hypertension,4,,Syn
4,601656,162393,2018-04-08,discharge diagnosis,icd-10-cm,R61,Generalized hyperhidrosis,5,,Syn


In [17]:
con.execute('select * from syntegra_ehr.encounter offset floor(random() * (select count(*) from syntegra_ehr.encounter)) limit 5').fetchdf()

,encounter_id,patient_id,encounter_type,encounter_start_date,encounter_end_date,admit_source_code,admit_source_description,admit_type_code,admit_type_description,discharge_disposition_code,discharge_disposition_description,physician_npi,location,facility_npi,ms_drg,paid_amount,data_source
0,875906,274022,other,2018-03-17 00:00:00,2018-03-17 00:00:00,,,,,07,Left against medical advice or discontinued care.,,,,,,Syn
1,594560,177195,emergency department,2018-05-09 00:00:00,2018-05-09 00:00:00,,,,,01,Discharged to home/self-care (routine charge),NONE,FXFC,,,,Syn
2,1031095,111857,emergency department,2018-10-25 00:00:00,2018-10-25 00:00:00,,,,,01,Discharged to home/self-care (routine charge),TW843,EC,,,,Syn
3,602959,6017,emergency department,2018-08-11 00:00:00,2018-08-11 00:00:00,,,,,1,,SMITPE,EDDEPSDC,,,,Syn
4,745203,56967,other,2018-01-05 00:00:00,2018-01-05 00:00:00,,,,,01,Discharged to home/self-care (routine charge),CC.HP,ERS,,,,Syn


In [18]:
con.execute('select * from syntegra_ehr.lab offset floor(random() * (select count(*) from syntegra_ehr.lab)) limit 5').fetchdf()

,encounter_id,patient_id,order_id,order_date,result_date,component_name,loinc,loinc_description,result,units,reference_range,specimen,data_source
0,637982,117706,,,2018-02-18,HEMATOCRIT,20570-8,HEMATOCRIT,39.1,%,37-47,,Syn
1,606832,281066,,,2018-09-28,Red Blood Count,789-8,Red Blood Count,3.53,10^6/ul,4.00-5.40,,Syn
2,606832,281066,,,2018-09-26,Red Blood Count,789-8,Red Blood Count,4.2,10^6/ul,4.00-5.40,,Syn
3,644445,303470,,,2018-08-28,Lymphocytes %,,,6.2,%,,,Syn
4,644445,303470,,,2018-08-30,Immature Granulocytes %,,,1.2,%,0-4,,Syn


In [19]:
con.execute('select * from syntegra_ehr.location offset floor(random() * (select count(*) from syntegra_ehr.location)) limit 5').fetchdf()

,facility_npi,facility_name,facility_type,hospital_type,parent_organization,data_source
0,,SCSULT,,,,
1,,M.MRI,,,,
2,,MTU,,,,
3,,6N,,,,
4,,PROCWHC3,,,,


In [20]:
con.execute('select * from syntegra_ehr.medication offset floor(random() * (select count(*) from syntegra_ehr.medication)) limit 5').fetchdf()

,encounter_id,patient_id,request_date,filled_date,paid_date,request_status,medication_name,ndc,rx_norm,dose,dose_unit,quantity,refills,duration,route,physician_npi,note,paid_amount,data_source
0,667950,143521,,,,active,THYR15TA,,,,,,,,PO,,,,Syn
1,667950,143521,2018-05-29,,,active,REMIFENTANIL HCL 1 MG VIAL,67457019803,,ONE,,,,,,,,,Syn
2,667950,143521,2018-05-29,,,active,PROPOFOL EMUL(*) 10MG/ML 20 ML 20 ML,63323026920,,ONE,,,,,,,,,Syn
3,667950,143521,2018-05-29,,,active,METO-259,00378459610,,,,,,,PO,,,,Syn
4,667950,143521,,,,active,LISI20TA29,00143971301,,,,,,,PO,,,,Syn


In [21]:
con.execute('select * from syntegra_ehr.patient offset floor(random() * (select count(*) from syntegra_ehr.patient)) limit 5').fetchdf()

,patient_id,name,gender,race,ethnicity,birth_date,death_date,death_flag,address,city,state,zip_code,phone,email,ssn,data_source
0,311254,Paxton Bishop,male,Native Hawaiian or Other Pacific Islander,,1973-02-08,2018-12-20,1,311 Fake St,Clute,TX,77531,555-555-1254,Paxton.Bishop@example.com,,Syn
1,88665,Zaid Morales,male,White,,1982-12-16,,,886 Fake St,Evans Mills,NY,13637,555-555-8665,Zaid.Morales@example.com,,Syn
2,205220,Knox Bryant,male,White,,1952-02-09,,,205 Fake St,De Kalb Junction,NY,13630,555-555-5220,Knox.Bryant@example.com,,Syn
3,168879,Maleah Kennedy,female,White,,1954-06-12,,,168 Fake St,Saint George,KS,66535,555-555-8879,Maleah.Kennedy@example.com,,Syn
4,351778,Kylian Soto,male,White,,1964-12-19,,,351 Fake St,,,,555-555-1778,Kylian.Soto@example.com,,Syn


In [22]:
con.execute('select * from syntegra_ehr.practitioner offset floor(random() * (select count(*) from syntegra_ehr.practitioner)) limit 5').fetchdf()

,physician_npi,name,specialty,sub_specialty,data_source
0,HAHNM,HAHNM,,,Syn
1,COUGTI,COUGTI,,,Syn
2,MIC0252,MIC0252,,,Syn
3,WLIGHTBU,WLIGHTBU,,,Syn
4,CARSE,CARSE,,,Syn


In [23]:
con.execute('select * from syntegra_ehr.procedure offset floor(random() * (select count(*) from syntegra_ehr.procedure)) limit 5').fetchdf()

,encounter_id,patient_id,procedure_date,code_type,code,description,physician_npi,data_source
0,836385,163148,2018-06-03,icd-10-cm,0DB98ZX,"Excision of Duodenum, Via Natural or Artificia...",KOZOR,Syn
1,669028,147377,2018-12-30,icd-10-cm,4A1HXCZ,"Monitoring of Products of Conception, Cardiac ...",POUANTOC,Syn
2,1016525,197801,2018-02-02,icd-10-cm,0DB68ZX,"Excision of Stomach, Via Natural or Artificial...",LEOAN,Syn
3,1016525,197801,2018-02-02,icd-10-cm,0DB58ZX,"Excision of Esophagus, Via Natural or Artifici...",LEOAN,Syn
4,935511,121640,2018-01-30,icd-10-cm,10E0XZZ,"Delivery of Products of Conception, External A...",MCQUGE,Syn


In [24]:
con.execute('select * from syntegra_ehr.vital_sign offset floor(random() * (select count(*) from syntegra_ehr.vital_sign)) limit 5').fetchdf()

,encounter_id,patient_id,component_id,loinc,loinc_description,vital_date,value,units,data_source
0,820024,78881,,8310-5,Body temperature,2018-04-29,98.0,degF,Syn
1,638402,349213,,2708-6,Oxygen saturation in Arterial blood,2018-03-24,98.0,%O2,Syn
2,602959,6017,,8867-4,Heart Rate,2018-08-11,68.0,beats/min,Syn
3,820024,78881,,8867-4,Heart Rate,2018-04-28,71.0,beats/min,Syn
4,776710,33776,,2708-6,Oxygen saturation in Arterial blood,2018-02-15,99.0,%O2,Syn


In [25]:
con.close()